### Example 1: 2 node system

#### Importing packages

In [117]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import cartopy as ccrs
plt.style.use('bmh')
import os

#### Step 1:Fuel & Powerplant data(Costs, efficiency)

In [118]:
costs = pd.read_csv("..\data\costs_Ambon.csv", index_col=[0,1])

In [120]:
costs.head(2)

,,CO2 intensity,FOM,VOM,discount rate,efficiency,fuel,investment,lifetime
technology,year,,,,,,,,
CCGT,2030,0.187,2.50,4.0,0.07,0.50,0.0,800000,30
OCGT,2030,0.187,3.75,3.0,0.07,0.39,0.0,400000,30


costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs.unit = costs.unit.str.replace("/kW", "/MW")

costs = costs.value.unstack().fillna(defaults)

In [121]:
def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

In [ ]:
costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

In [ ]:
annuity = costs.apply(lambda x: annuity(x["discount rate"], x["lifetime"]), axis=1)

In [ ]:
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

#### Loading time series data

In [ ]:
timeseries = pd.read_csv("../data/timeseries_Ambon.csv", index_col=0, parse_dates=True)

In [ ]:
os.getcwd()

In [ ]:
timeseries

In [ ]:
resolution = 4
ts = ts.resample(f"{resolution}H").first()

In [ ]:
ts['onwind']

#### Step 2: Model Initialization

In [ ]:
# A) Create a network object
n=pypsa.Network()

In [ ]:
# B) Add buses
n.add("Bus", "electricity")
#n.add("Bus", "Ambon_1", y=13.4, x=52.5, v_nom=400, carrier='AC')#Ambon_1
#n.add("Bus", "Ambon_2", y=14.3, x=51.7, v_nom=400, carrier='AC') #Ambon_2

In [ ]:
# C) Add snapshots to the network
n.set_snapshots(ts.index)

In [ ]:
n.snapshots

In [ ]:
n.snapshot_weightings.head(10)

In [ ]:
n.snapshot_weightings.loc[:,:] = resolution

In [ ]:
n.snapshot_weightings.head(10)

In [ ]:
# D) Add carriers with emissions associated with it
carriers = ["onwind", "offwind", "solar", "OCGT", "hydrogen storage underground", "battery storage"]
n.madd(
    "Carrier",
    carriers, 
    color=["dodgerblue", "aquamarine", "gold", "indianred", "magenta", "yellowgreen"],
    co2_emissions=[costs.at[c, "CO2 intensity"] for c in carriers]
)

In [ ]:
# E) Add demand time series
n.add(
    "Load",
    "demand",
    bus="electricity",
    p_set=ts.load,
)

In [ ]:
n.loads_t.p_set.plot(figsize=(9,3), ylabel="MW")

In [ ]:
# F) Add generators #dispatchable
n.add(
    "Generator",
    "OCGT",
    bus='electricity',
    carrier="OCGT",
    capital_cost=costs.at["OCGT", "capital_cost"],
    marginal_cost=costs.at["OCGT", "marginal_cost"],
    efficiency=costs.at["OCGT", "efficiency"],
    p_nom_extendable=True,
)

In [ ]:
# G) Add generators Renewable
for tech in ["onwind", "offwind", "solar"]:
    n.add(
        "Generator",
        tech,
        bus='electricity',
        carrier=tech,
        p_max_pu=ts[tech],
        capital_cost=costs.at[tech, "capital_cost"],
        marginal_cost=costs.at[tech, "marginal_cost"],
        efficiency=costs.at[tech, "efficiency"],
        p_nom_extendable=True,
    )

In [ ]:
n.generators_t.p_max_pu.loc["2015-07"].plot(figsize=(6,2), ylabel="CF")

#### Step 3) Run the model and print the optimization results

##### A) Solve the model

In [ ]:
n.lopf(solver_name='glpk')

##### B) Optimized capacity (GW) & Energy generation(TWh)

In [ ]:
# Optimized capacity in GW
n.generators.p_nom_opt.div(1e3)

In [ ]:
# The total energy generation by technology in GW:
n.snapshot_weightings.generators @ n.generators_t.p.div(1e6) # TWh

##### C) Print various Costs ( Total, capex, opex)

In [ ]:
# The total system cost in billion Euros per year:
n.objective / 1e9

In [ ]:
# Costs M€/a
opex = n.snapshot_weightings.generators @ (n.generators_t.p * n.generators.marginal_cost).div(1e6) # M€/a
capex = (n.generators.p_nom_opt * n.generators.capital_cost).div(1e6) # M€/a
print(capex, opex) # M€/a

In [ ]:
#Total system cost by technology M€/a
(n.statistics.capex() + n.statistics.opex(aggregate_time='sum')).div(1e6)

##

##### D) Emissions (t/h)

In [ ]:
emissions = n.generators_t.p / n.generators.efficiency * n.generators.carrier.map(n.carriers.co2_emissions) # t/h

In [ ]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6) # Mt

#### E) Plotting optimal dispatch

This function takes the network object `n` as an argument and, optionally, a time frame. We want to plot the load time series, and stacked area charts for electricity feed-in and storage charging. Technologies should be coloured by their color defined in `n.carriers`.

In [ ]:
def plot_dispatch(n, time="2015-07"):
    
    p_by_carrier = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum().div(1e3)
    
    if not n.storage_units.empty:
        sto = n.storage_units_t.p.groupby(n.storage_units.carrier, axis=1).sum().div(1e3)
        p_by_carrier = pd.concat([p_by_carrier, sto], axis=1)
    
    fig, ax = plt.subplots(figsize=(6, 3))
    
    color = p_by_carrier.columns.map(n.carriers.color)
    
    p_by_carrier.where(p_by_carrier>0).loc[time].plot.area(
        ax=ax,
        linewidth=0,
        color=color,
    )
    
    charge = p_by_carrier.where(p_by_carrier<0).dropna(how='all', axis=1).loc[time]
    
    if not charge.empty:

        charge.plot.area(
            ax=ax,
            linewidth=0,
            color=charge.columns.map(n.carriers.color),
        )
    
    n.loads_t.p_set.sum(axis=1).loc[time].div(1e3).plot(ax=ax, c='k')

    plt.legend(loc=(1.05,0))
    ax.set_ylabel("GW")
    ax.set_ylim(-200, 200)

In [ ]:
plot_dispatch(n)

#### F) Add Storage units ( Battery(Energy-to-power ratio:6 hours), Hydrogen) 

In [ ]:
n.add(
    "StorageUnit",
    "battery storage",
    bus='electricity',
    carrier="battery storage",
    max_hours=6,
    capital_cost=costs.at["battery inverter", "capital_cost"] + 6 * costs.at["battery storage", "capital_cost"],
    efficiency_store=costs.at["battery inverter", "efficiency"],
    efficiency_dispatch=costs.at["battery inverter", "efficiency"],
    p_nom_extendable=True,
    cyclic_state_of_charge=True,
)

Second, the hydrogen storage. This one is composed of an electrolysis to convert electricity to hydrogen, a fuel cell to re-convert hydrogen to electricity and underground storage (e.g. in salt caverns). We assume an energy-to-power ratio of 168 hours, such that this type of storage can be used for weekly balancing.

In [ ]:
n.plot(bus_sizes=0.01, margin=3);

In [ ]:
n.generators_t.p

In [ ]:
n.buses_t.marginal_price

In [ ]:
s = n.loads.groupby('bus').p_set.sum() / 1e4

In [ ]:
n.generators_t.p.loc["now"]

In [ ]:
n.generators.groupby("carrier").p_nom.sum().div(1e3).plot.bar()
plt.ylabel('GW')